# Spotify API Data Collection

In this project, we are extracting data through the Spotify API and converting it into a dataframe to make it ready for data manipulation. During this process, we personalize our search by exploring the Spotify API's search function.

In [8]:
#Importing the libraries

import requests
import pandas as pd
import json
import time

# Spotify API credentials
To access the Spotify API, we need to authenticate using a client ID and client secret. These credentials allow us to obtain an access token, which is required to make requests to various Spotify API endpoints. The TOKEN_URL is used to obtain this access token.

In [ ]:

CLIENT_ID = "your_id"
CLIENT_SECRET = "your_secret"
TOKEN_URL = "your token"


# Obtaining an access token from Spotify API
This function sends a POST request to the Spotify authentication endpoint with the necessary credentials. If the request is successful, it returns an access token that will be used in subsequent API requests. In case of an error, it prints the error message and returns None.


In [10]:

def get_access_token():
    try:
        response = requests.post(TOKEN_URL, {
            'grant_type': 'client_credentials',
            'client_id': CLIENT_ID,
            'client_secret': CLIENT_SECRET
        })
        response.raise_for_status()  # Hata kontrolü
        token_info = response.json()
        return token_info.get('access_token')
    except requests.exceptions.RequestException as e:
        print(f"Error fetching token: {e}")
        return None

# Fetching data from any Spotify API endpoint
This function takes an API endpoint URL and an optional parameter dictionary. It first retrieves an access token and then uses it to send an authenticated GET request to Spotify. The function returns the JSON response from the API.

In [11]:

def get_spotify_data(endpoint, params=None):
    token = get_access_token()
    if token is None:
        print("Unable to fetch access token.")
        return {}
    
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(endpoint, headers=headers, params=params)
    return response.json()

#  Fetching newly released albums from Spotify
This function specifically calls the 'new-releases' API endpoint to retrieve the latest album releases. The response contains album details such as name, artist, and release date.

In [12]:

def fetch_new_albums():
    url = "https://api.spotify.com/v1/browse/new-releases"
    data = get_spotify_data(url)
    return data

# Processing the retrieved data and convert it into a structured Pandas DataFrame
The function extracts album details such as album name, artist name, and release date and saves it to a CSV file


In [17]:

def process_data(data):
    if 'albums' not in data or 'items' not in data['albums']:
        print("No albums found.")
        return pd.DataFrame()
    
    albums = data['albums']['items']
    df = pd.DataFrame([{
        "album_name": album["name"],
        "artist_name": album["artists"][0]["name"],
        "release_date": album["release_date"]
    } for album in albums])
    return df


def save_to_csv(df, filename="new_released_albums.csv"):
    if not df.empty:
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    else:
        print("No data to save.")

Main function to execute the data fetching, processing, and saving operations 

This function calls the appropriate functions in sequence to fetch new tracks, process them,and save the results to a CSV file.

In [18]:

def main():
    data = fetch_new_albums()
    df = process_data(data)
    save_to_csv(df)

if __name__ == "__main__":
    main()


Data saved to new_released_albums.csv


In [19]:
# Reading a CSV file into a Pandas DataFrame
import pandas as pd

# CSV dosyasını oku
df = pd.read_csv('new_released_albums.csv', sep=',')

df

,album_name,artist_name,release_date
0,MUSIC,Playboi Carti,2025-03-14
1,The Father of Make Believe,Coheed and Cambria,2025-03-14
2,MAYHEM,Lady Gaga,2025-03-07
3,Ruby,JENNIE,2025-03-05
4,One More Trip Around The Sun,Cari Cari,2025-03-07
5,Wir verglühen,Lena&Linus,2025-03-07
6,"Alles, was ich nie sagen konnte",CCOSMO,2025-03-07
7,More Leaks,YoungBoy Never Broke Again,2025-03-07
8,Stell dir vor,Ritter Lean,2025-02-28
9,Oldschool Love,Leony,2025-02-28


Retrieving artists based on a specific genre

In [20]:
def get_artists_by_genre(genre, limit=50):
    try:
        token = get_access_token()
        if not token:
            raise ValueError("Token could not be retrieved.")

        headers = {"Authorization": f"Bearer {token}"}
        url = "https://api.spotify.com/v1/search"
        
        params = {
            "q": f"genre:{genre}",
            "type": "artist",
            "limit": limit
        }

        response = requests.get(url, headers=headers, params=params)
        
        # Check if the API response was successful
        response.raise_for_status()  # Raises HTTPError for failed requests
        data = response.json()

        # Extract and return the list of artists from the API response
        return data["artists"]["items"]

    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the API request: {e}")
    except ValueError as e:
        print(f"Token error: {e}")
    except KeyError as e:
        print(f"Expected data missing: {e}")
    except Exception as e:
        print(f"An unknown error occurred: {e}")


In [21]:
def create_artist_dataframe(genre):
    try:
        artists = get_artists_by_genre(genre)
        if not artists:
            raise ValueError("No artists found for the given genre.")
        
        artist_list = []
        for i, artist in enumerate(artists, start=1):
            try:
                artist_list.append({
                    "index": i,
                    "name": artist.get("name", "Unknown"),
                    "popularity": artist.get("popularity", 0),
                    "followers": artist.get("followers", {}).get("total", 0)
                })
            except Exception as e:
                print(f"Error processing artist data: {e}")
                continue
        
        df = pd.DataFrame(artist_list)
        return df
    except Exception as e:
        print(f"Error fetching artists: {e}")
        return pd.DataFrame()

# Fetch rock artists and create a DataFrame
# The following code fetches data for artists in the "rock" genre and stores it in a Pandas DataFrame.

rock_df = create_artist_dataframe("rock")
rock_df


,index,name,popularity,followers
0,1,Gene Rockwell,23,11686
1,2,Genki Rockets,21,16437
2,3,Linkin Park,89,29167497
3,4,Imagine Dragons,87,56689881
4,5,Rammstein,80,11037537
5,6,OneRepublic,83,18039912
6,7,Böhse Onkelz,68,1342794
7,8,Arctic Monkeys,87,29313571
8,9,The Neighbourhood,84,19826332
9,10,Queen,84,53265566
